<a href="https://colab.research.google.com/github/fxgurv/YASGU/blob/master/aishorts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/whisper.git

In [ ]:
!pip install ffmpeg-python==0.2.0

In [ ]:
!apt install imagemagick

In [ ]:
!pip install moviepy==2.0.0.dev2
!pip install imageio==2.25.1

In [ ]:
!pip install pyttsx3

In [ ]:
!pip install pocketsphinx

In [ ]:
!pip install -U g4f

In [ ]:
!pip install gTTS

In [ ]:
!pip install gdown

In [ ]:
import ast
import os
import re
import whisper
import json
import pyttsx3
import g4f
from gtts import gTTS
from pocketsphinx import AudioFile
from datetime import datetime

In [ ]:
from moviepy.editor import *
import random
import os
import json

from moviepy.editor import TextClip, CompositeVideoClip, ColorClip
import numpy as np
directory_path = 'template_videos/'

In [ ]:
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

In [ ]:
!mkdir results
!mkdir temp_audio
!gdown --folder https://drive.google.com/drive/folders/1Dg-1GFfkLrTZz76zPLXHMdMVvOb54VgV

In [ ]:
def split_text_into_lines(data):
    MaxChars = 80
    # maxduration in seconds
    MaxDuration = 3.0
    # Split if nothing is spoken (gap) for these many seconds
    MaxGap = 1.5

    subtitles = []
    line = []
    line_duration = 0
    line_chars = 0

    for idx, word_data in enumerate(data):
        word = word_data["word"]
        start = word_data["start"]
        end = word_data["end"]

        line.append(word_data)
        line_duration += end - start

        temp = " ".join(item["word"] for item in line)

        # Check if adding a new word exceeds the maximum character count or duration
        new_line_chars = len(temp)

        duration_exceeded = line_duration > MaxDuration
        chars_exceeded = new_line_chars > MaxChars
        if idx > 0:
            gap = word_data['start'] - data[idx - 1]['end']
            # print (word,start,end,gap)
            maxgap_exceeded = gap > MaxGap
        else:
            maxgap_exceeded = False

        if duration_exceeded or chars_exceeded or maxgap_exceeded:
            if line:
                subtitle_line = {
                    "word": " ".join(item["word"] for item in line),
                    "start": line[0]["start"],
                    "end": line[-1]["end"],
                    "textcontents": line
                }
                subtitles.append(subtitle_line)
                line = []
                line_duration = 0
                line_chars = 0

    if line:
        subtitle_line = {
            "word": " ".join(item["word"] for item in line),
            "start": line[0]["start"],
            "end": line[-1]["end"],
            "textcontents": line
        }
        subtitles.append(subtitle_line)

    return subtitles

In [ ]:
def create_caption(textJSON, framesize, font="Candy-Beans", fontsize=65, color='white', bgcolor='blue'):
    wordcount = len(textJSON['textcontents'])
    full_duration = textJSON['end'] - textJSON['start']

    word_clips = []
    xy_textclips_positions = []

    x_pos = 0
    y_pos = 800

    frame_width = framesize[0]
    frame_height = framesize[1]
    x_buffer = frame_width * 1 / 10
    y_buffer = frame_height * 1 / 5

    space_width = ""
    space_height = ""

    for index, wordJSON in enumerate(textJSON['textcontents']):
        duration = wordJSON['end'] - wordJSON['start']
        word_clip = TextClip(wordJSON['word'], font=font, fontsize=fontsize, color=color).set_start(
            textJSON['start']).set_duration(full_duration)
        word_clip_space = TextClip(" ", font=font, fontsize=fontsize, color=color).set_start(
            textJSON['start']).set_duration(full_duration)
        word_width, word_height = word_clip.size
        space_width, space_height = word_clip_space.size
        if x_pos + word_width + space_width > frame_width - 2 * x_buffer:
            # Move to the next line
            x_pos = 0
            y_pos = y_pos + word_height + 40

            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos": x_pos + x_buffer,
                "y_pos": y_pos + y_buffer,
                "width": word_width,
                "height": word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.set_position((x_pos + x_buffer, y_pos + y_buffer))
            word_clip_space = word_clip_space.set_position((x_pos + word_width + x_buffer, y_pos + y_buffer))
            x_pos = word_width + space_width
        else:
            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos": x_pos + x_buffer,
                "y_pos": y_pos + y_buffer,
                "width": word_width,
                "height": word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.set_position((x_pos + x_buffer, y_pos + y_buffer))
            word_clip_space = word_clip_space.set_position((x_pos + word_width + x_buffer, y_pos + y_buffer))

            x_pos = x_pos + word_width + space_width

        word_clips.append(word_clip)
        word_clips.append(word_clip_space)

    for highlight_word in xy_textclips_positions:
        word_clip_highlight = TextClip(highlight_word['word'], font=font, fontsize=fontsize, color='LightGreen').set_start(highlight_word['start']).set_duration(
            highlight_word['duration'])
        word_clip_highlight = word_clip_highlight.set_position((highlight_word['x_pos'], highlight_word['y_pos']))
        word_clips.append(word_clip_highlight)

    return word_clips

In [ ]:
def new_video(audioPath, fileName):
    frame_size = (1080, 1920)
    with open(f'{audioPath}.json', 'r') as f:
        wordlevel_info_modified = json.load(f)

    linelevel_subtitles = split_text_into_lines(wordlevel_info_modified)

    all_linelevel_splits = []

    for line in linelevel_subtitles:
        out = create_caption(line, frame_size)
        all_linelevel_splits.extend(out)

    files = os.listdir(directory_path)
    random_file = random.choice(files)
    random_file_path = os.path.join(directory_path, random_file)

    video_path = random_file_path
    audio_path = f'{audioPath}.mp3'

    video_clip = VideoFileClip(video_path).fx(afx.audio_normalize)
    video_clip.audio = video_clip.audio.volumex(0.04)
    audio_clip = AudioFileClip(audio_path).volumex(1.25)

    video_start_time = random.uniform(0, video_clip.duration - audio_clip.duration)

    video_clip = video_clip.subclip(video_start_time, video_start_time + audio_clip.duration)

    new_audioclip = CompositeAudioClip([audio_clip, video_clip.audio])
    video_clip.audio = new_audioclip

    video_clip = CompositeVideoClip([video_clip] + all_linelevel_splits)

    output_path = f'results/{fileName}.mp4'
    video_clip.write_videofile(output_path, codec='libx264')

    # Zwolnij zasoby
    video_clip.close()
    audio_clip.close()

In [ ]:
def extract_tale_from_string(text):
    return ast.literal_eval(re.search('({.+})', text).group(0))

In [ ]:
g4f.debug.logging = True # enable logging
g4f.check_version = False # Disable automatic version checking
print(g4f.version) # check version
print(g4f.Provider.Ails.params)  # supported args

# Automatic selection of provider
language = 'en'

n = 3

In [ ]:
from g4f.Provider import (
    AItianhu,
    Aichat,
    Bard,
    Bing,
    ChatBase,
    ChatgptAi,
    OpenaiChat,
    Vercel,
    You,
    Yqcloud,
)

In [ ]:
for i in range(n):
  response = g4f.ChatCompletion.create(
      model="gpt-3.5-turbo",
      provider=	g4f.Provider.GptGo,
      temperature=0.9,
      messages=[{"role": "user",
                "content": "tell a viral 420 chars dark viral joke (random) about mother with unexpected plot twist in exactly format!!! (without anything more): {'title' : 'Title of Tale', 'content' : 'Tale'}"}]
  )
  for message in response:
      print(message, flush=True, end='')

  response = extract_tale_from_string("".join(str(a) for a in response))


  myobj = gTTS(text=response['content'], lang=language, tld='us')
  now = datetime.now()

  myobj.save(f"temp_audio/{response['title']}.mp3")

  model = whisper.load_model("medium")
  result = model.transcribe(f"temp_audio/{response['title']}.mp3", word_timestamps=True)

  wordlevel_info = []

  for each in result['segments']:
      words = each['words']
      for word in words:
          # print (word['word'], "  ",word['start']," - ",word['end'])
          wordlevel_info.append({'word': word['word'].strip(), 'start': word['start'], 'end': word['end']})

  with open(f"temp_audio/{response['title']}.json", 'w') as f:
      json.dump(wordlevel_info, f, indent=4)

  new_video(f"temp_audio/{response['title']}", f"{response['title']}")